In [13]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, f1_score
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [2]:
df_raw = pd.read_csv('/kaggle/input/leopard-challenge-classification/train.csv')
test = pd.read_csv(r'/kaggle/input/leopard-challenge-classification/test.csv')
df_raw.shape, test.shape

((13863, 26), (5942, 25))

In [3]:
test_cv = test.drop(['oral', 'ID'], axis = 1).copy()
test_cv['tartar'] = test_cv['tartar'].apply(lambda x: 1 if x == 'Y' else 0)
merged_data = df_raw.drop(['oral', 'ID'], axis=1).copy()
merged_data['tartar'] = merged_data['tartar'].apply(lambda x: 1 if x == 'Y' else 0)

In [4]:
# Создание бинарного признака на основе BMI
merged_data['BMI'] = merged_data['weight(kg)'] / ((merged_data['height(cm)'] / 100) ** 2)
merged_data['BMI_status'] = (merged_data['BMI'] < 18.5) | (merged_data['BMI'] >= 24.9)
merged_data['BMI_status'] = merged_data['BMI_status'].astype(int)
merged_data['log_tr'] = merged_data['triglyceride'].apply(np.log)
merged_data['log_gtp'] = merged_data['Gtp'].apply(np.log)
merged_data['log_log_alt'] = np.log(np.log(merged_data['ALT']))
merged_data['BMI_log_gtp'] = merged_data['BMI'].apply(np.log)
merged_data = merged_data.drop(['triglyceride', 'Gtp', 'ALT', 'BMI'], axis=1)

test_cv['BMI'] = test_cv['weight(kg)'] / ((test_cv['height(cm)'] / 100) ** 2)
test_cv['BMI_status'] = (test_cv['BMI'] < 18.5) | (test_cv['BMI'] >= 24.9)
test_cv['BMI_status'] = test_cv['BMI_status'].astype(int)
test_cv['log_tr'] = test_cv['triglyceride'].apply(np.log)
test_cv['log_gtp'] = test_cv['Gtp'].apply(np.log)
test_cv['log_log_alt'] = np.log(np.log(test_cv['ALT']))
test_cv['BMI_log_gtp'] = test_cv['BMI'].apply(np.log)
test_cv = test_cv.drop(['triglyceride', 'Gtp', 'ALT', 'BMI'], axis=1)

In [6]:
X = merged_data.drop('smoking', axis=1)
y = merged_data['smoking']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [9]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [20]:
import optuna

def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.15),
        'n_estimators': trial.suggest_int('n_estimators', 800, 2500),
        'max_depth': trial.suggest_int('max_depth', 3, 11),
        'subsample': trial.suggest_float('subsample', 0.5, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'scale_pos_weight': 1/y_train.mean(),
        'early_stopping_rounds': 70,
        'eval_metric': "auc"
    }
    
    f1_scores = []
    
    for train_index, val_index in kf.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        model = XGBClassifier(**params)
        eval_set = [(X_val_fold, y_val_fold)]
        model.fit(X_train_fold, y_train_fold, eval_set=eval_set, verbose=100) # , early_stopping_rounds=50

        y_prob_fold = model.predict_proba(X_val_fold)[:, 1]
        thresholds = np.linspace(0, 1, 300)
        fold_f1_scores = [f1_score(y_val_fold, y_prob_fold > thresh) for thresh in thresholds]
        f1_scores.append(max(fold_f1_scores))
    
    return -np.mean(f1_scores)  # Среднее значение F1 по всем фолдам

# Запуск оптимизации Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=25)  

# Лучшие параметры
best_params = study.best_params

print("Best parameters:", best_params)
print("Best F1 score:", -study.best_value)


[I 2023-10-12 15:08:37,000] A new study created in memory with name: no-name-d62394bd-36d1-4fe6-8584-10464161d096


[0]	validation_0-auc:0.56958
[100]	validation_0-auc:0.66614
[200]	validation_0-auc:0.67344
[264]	validation_0-auc:0.67391
[0]	validation_0-auc:0.60569
[100]	validation_0-auc:0.70013
[115]	validation_0-auc:0.69749
[0]	validation_0-auc:0.62321
[100]	validation_0-auc:0.70597
[200]	validation_0-auc:0.71262
[249]	validation_0-auc:0.71439
[0]	validation_0-auc:0.59120
[100]	validation_0-auc:0.68463
[200]	validation_0-auc:0.70051
[300]	validation_0-auc:0.70127
[335]	validation_0-auc:0.70367
[0]	validation_0-auc:0.61434
[100]	validation_0-auc:0.70012
[165]	validation_0-auc:0.70108


[I 2023-10-12 15:09:01,105] Trial 0 finished with value: -0.43543834672647935 and parameters: {'learning_rate': 0.13213251588820324, 'n_estimators': 2108, 'max_depth': 10, 'subsample': 0.5152078440520851, 'colsample_bytree': 0.7294807802672215, 'gamma': 0.0885468423472584}. Best is trial 0 with value: -0.43543834672647935.


[0]	validation_0-auc:0.61412
[100]	validation_0-auc:0.67456
[200]	validation_0-auc:0.67611
[300]	validation_0-auc:0.67920
[400]	validation_0-auc:0.68363
[500]	validation_0-auc:0.68558
[600]	validation_0-auc:0.68728
[689]	validation_0-auc:0.68664
[0]	validation_0-auc:0.59322
[100]	validation_0-auc:0.70408
[200]	validation_0-auc:0.71303
[269]	validation_0-auc:0.71233
[0]	validation_0-auc:0.59717
[100]	validation_0-auc:0.71513
[200]	validation_0-auc:0.72121
[300]	validation_0-auc:0.72378
[400]	validation_0-auc:0.72681
[500]	validation_0-auc:0.72858
[600]	validation_0-auc:0.72965
[641]	validation_0-auc:0.72923
[0]	validation_0-auc:0.58400
[100]	validation_0-auc:0.69961
[200]	validation_0-auc:0.70922
[300]	validation_0-auc:0.71398
[400]	validation_0-auc:0.71842
[494]	validation_0-auc:0.71804
[0]	validation_0-auc:0.59100
[100]	validation_0-auc:0.71373
[200]	validation_0-auc:0.72101
[300]	validation_0-auc:0.72455
[380]	validation_0-auc:0.72246


[I 2023-10-12 15:10:00,777] Trial 1 finished with value: -0.44987852734499273 and parameters: {'learning_rate': 0.04142494487977635, 'n_estimators': 1653, 'max_depth': 11, 'subsample': 0.5628148240547551, 'colsample_bytree': 0.9884625987373434, 'gamma': 0.22355658732139855}. Best is trial 1 with value: -0.44987852734499273.


...

[0]	validation_0-auc:0.58700
[100]	validation_0-auc:0.68209
[200]	validation_0-auc:0.68862
[278]	validation_0-auc:0.68855
[0]	validation_0-auc:0.61913
[100]	validation_0-auc:0.71307
[200]	validation_0-auc:0.71927
[300]	validation_0-auc:0.72224
[371]	validation_0-auc:0.72151
[0]	validation_0-auc:0.64751
[100]	validation_0-auc:0.73500
[200]	validation_0-auc:0.73552
[221]	validation_0-auc:0.73538
[0]	validation_0-auc:0.58973
[100]	validation_0-auc:0.69868
[200]	validation_0-auc:0.70718
[300]	validation_0-auc:0.71127
[382]	validation_0-auc:0.71171
[0]	validation_0-auc:0.59871
[100]	validation_0-auc:0.71532
[200]	validation_0-auc:0.72170
[300]	validation_0-auc:0.72593
[400]	validation_0-auc:0.72680
[500]	validation_0-auc:0.72667
[561]	validation_0-auc:0.72561


[I 2023-10-12 15:19:05,315] Trial 23 finished with value: -0.45047012711586953 and parameters: {'learning_rate': 0.07081031418927757, 'n_estimators': 1611, 'max_depth': 11, 'subsample': 0.9103061578166558, 'colsample_bytree': 0.5626692450624043, 'gamma': 0.2976032642899215}. Best is trial 14 with value: -0.4573087610687489.


[0]	validation_0-auc:0.58551
[100]	validation_0-auc:0.67793
[200]	validation_0-auc:0.68961
[300]	validation_0-auc:0.69124
[332]	validation_0-auc:0.69084
[0]	validation_0-auc:0.62598
[100]	validation_0-auc:0.71826
[200]	validation_0-auc:0.72327
[292]	validation_0-auc:0.72388
[0]	validation_0-auc:0.59298
[100]	validation_0-auc:0.71057
[200]	validation_0-auc:0.72108
[300]	validation_0-auc:0.72435
[400]	validation_0-auc:0.72451
[458]	validation_0-auc:0.72354
[0]	validation_0-auc:0.60482
[100]	validation_0-auc:0.70017
[200]	validation_0-auc:0.71157
[300]	validation_0-auc:0.71616
[400]	validation_0-auc:0.71835
[500]	validation_0-auc:0.72059
[552]	validation_0-auc:0.72032
[0]	validation_0-auc:0.61720
[100]	validation_0-auc:0.71173
[200]	validation_0-auc:0.71508
[300]	validation_0-auc:0.71737
[354]	validation_0-auc:0.71631


[I 2023-10-12 15:19:37,327] Trial 24 finished with value: -0.4512647395849144 and parameters: {'learning_rate': 0.0885341418812912, 'n_estimators': 1046, 'max_depth': 10, 'subsample': 0.8517441031732926, 'colsample_bytree': 0.5700818516850485, 'gamma': 0.255117560957859}. Best is trial 14 with value: -0.4573087610687489.


Best parameters: {'learning_rate': 0.09101000886909859, 'n_estimators': 1199, 'max_depth': 11, 'subsample': 0.7635921298553625, 'colsample_bytree': 0.6682734311264994, 'gamma': 0.2921663159248171}
Best F1 score: 0.4573087610687489


In [21]:
# Обучение с лучшими гиперпараметрами на всем тренировочном наборе данных
best_model = XGBClassifier(**best_params, eval_metric='auc')
best_model.fit(X_train, y_train)

# Предсказания на тестовой выборке
y_prob_test = best_model.predict_proba(X_test)[:, 1]

# Определение оптимального порога
thresholds = np.linspace(0, 1, 300)
f1_scores_test = [f1_score(y_test, y_prob_test > thresh) for thresh in thresholds]
optimal_threshold_test = thresholds[np.argmax(f1_scores_test)]

# Применение оптимального порога для классификации
y_pred_test = [1 if prob >= optimal_threshold_test else 0 for prob in y_prob_test]

# Оценка качества модели с новым порогом на тестовой выборке
print("ROC AUC:", roc_auc_score(y_test, y_prob_test))
print("Average Precision:", average_precision_score(y_test, y_prob_test))
print("\nOptimal Threshold:", optimal_threshold_test)
print("\nClassification Report with Optimal Threshold:\n", classification_report(y_test, y_pred_test))

ROC AUC: 0.7358805112646052
Average Precision: 0.4476910804488899

Optimal Threshold: 0.06354515050167224

Classification Report with Optimal Threshold:
               precision    recall  f1-score   support

           0       0.89      0.71      0.79      2213
           1       0.36      0.65      0.47       560

    accuracy                           0.70      2773
   macro avg       0.63      0.68      0.63      2773
weighted avg       0.78      0.70      0.73      2773



In [22]:
import pickle

# Сохранение модели в файл
with open("best_xgb_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

    
# Загрузка модели из файла
# with open("best_xgb_model.pkl", "rb") as f:
#     loaded_model = pickle.load(f)
    

In [23]:
# Сохранение гиперпараметров в файл
with open("best_params.pkl", "wb") as f:
    pickle.dump(best_params, f)
    
# Загрузка гиперпараметров из файла
# with open("best_params.pkl", "rb") as f:
#     loaded_params = pickle.load(f)


In [ ]:
# Использование сохраненной модели
import pickle

# Загрузка модели из файла
with open("best_xgb_model.pkl", "rb") as f:
    loaded_model = pickle.load(f)

# Предсказания с помощью загруженной модели
predictions = loaded_model.predict(new_data)  # где new_data - новые данные, на которых вы хотите сделать предсказания

#  Использование списка лучших гиперпараметров

import pickle
from xgboost import XGBClassifier

# Загрузка гиперпараметров из файла
with open("best_params.pkl", "rb") as f:
    loaded_params = pickle.load(f)

# Создание и обучение модели с загруженными гиперпараметрами
model_with_loaded_params = XGBClassifier(**loaded_params)
model_with_loaded_params.fit(X_train, y_train)

# Предсказания с помощью обученной модели
predictions = model_with_loaded_params.predict(new_data)
